In [1]:
import requests
import mysql.connector
from mysql.connector import Error

import mysql_creds

In [2]:
mysql_creds.host

'localhost'

In [3]:
# Set up the API call variables
game_data = []
year = '2021'
season_type = '02' 
max_game_ID = 1290

## Drop existing tables

In [4]:
try:
    connection = mysql.connector.connect(host=mysql_creds.host,
                                         database=mysql_creds.database,
                                         user=mysql_creds.user,
                                         password=mysql_creds.password)
    
    
    mySql_drop_Query = """DROP TABLE IF EXISTS team ;\
    DROP TABLE IF EXISTS division;\
    DROP TABLE IF EXISTS conference;"""
    
    
    cursor = connection.cursor()
    result = cursor.execute(mySql_drop_Query)
    print("existing tables dropped successfully ")

except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

existing tables dropped successfully 


### Conference and Division data

#### Create the conference table and add all of the tables

In [5]:
conf_info = requests.get(url='https://statsapi.web.nhl.com/api/v1/conferences')
conf_info = conf_info.json()
conf_info = conf_info['conferences']

In [6]:
try:
    connection = mysql.connector.connect(host=mysql_creds.host,
                                         database=mysql_creds.database,
                                         user=mysql_creds.user,
                                         password=mysql_creds.password)
    
    
    mySql_Create_Table_Query = """CREATE TABLE conference(Id int NOT NULL,\
    name varchar(20) NOT NULL,\
    abbreviation varchar(5) NOT NULL,\
    nameShort varchar(10) NOT NULL,\
    PRIMARY KEY (Id))"""
    
    
    cursor = connection.cursor()
    result = cursor.execute(mySql_Create_Table_Query)
    print("conference table created successfully ")

except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

conference table created successfully 
MySQL connection is closed


In [7]:
def create_conference_insert_statement(conf_stats):
    
    conf_id = conf_stats['id']
    conf_fullname = conf_stats['name']
    conf_abbr = conf_stats['abbreviation']
    conf_nameShort = conf_stats['shortName']
    
    insert_statement = """INSERT INTO conference(Id, name, abbreviation, nameShort)\
    VALUES ({}, '{}', '{}',  '{}')"""\
    .format(conf_id, conf_fullname, conf_abbr, conf_nameShort)
    
    return insert_statement
    
for conference in conf_info:
        
    try:
        connection = mysql.connector.connect(host=mysql_creds.host,
                                         database=mysql_creds.database,
                                         user=mysql_creds.user,
                                         password=mysql_creds.password)
        
        mySql_insert_query = create_conference_insert_statement(conference)

        cursor = connection.cursor()
        cursor.execute(mySql_insert_query)
        connection.commit()
        print("{}, inserted successfully into conference table".format(conference['name']))
        cursor.close()

    except mysql.connector.Error as error:
        print("Failed to insert record into Laptop table {}".format(error))

    finally:
        if connection.is_connected():
            connection.close()
#             print("MySQL connection is closed")



Eastern, inserted successfully into conference table
Western, inserted successfully into conference table


#### Division

In [8]:
div_info = requests.get(url='https://statsapi.web.nhl.com/api/v1/divisions')
div_info = div_info.json()
div_info = div_info['divisions']

In [9]:
try:
    connection = mysql.connector.connect(host=mysql_creds.host,
                                         database=mysql_creds.database,
                                         user=mysql_creds.user,
                                         password=mysql_creds.password)
    
    
    mySql_Create_Table_Query = """CREATE TABLE division(Id int NOT NULL,\
    name varchar(20) NOT NULL,\
    nameShort varchar(5) NOT NULL,\
    abbreviation varchar(5) NOT NULL,\
    conference int NOT NULL,\
    PRIMARY KEY (Id),\
    FOREIGN KEY (conference) REFERENCES conference(Id))"""
    
    
    cursor = connection.cursor()
    result = cursor.execute(mySql_Create_Table_Query)
    print("division Table created successfully ")

except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

division Table created successfully 
MySQL connection is closed


In [10]:
def create_division_insert_statement(div_info):
    
    div_id = div_info['id']
    div_fullname = div_info['name']
    div_abbr = div_info['abbreviation']
    div_nameShort = div_info['nameShort']
    div_conf = div_info['conference']['id']
    
    insert_statement = """INSERT INTO division(Id, name, abbreviation, nameShort, conference)\
    VALUES ({}, '{}', '{}',  '{}', {})"""\
    .format(div_id, div_fullname, div_abbr, div_nameShort, div_conf)
    
    return insert_statement
    
for division in div_info:
        
    try:
        connection = mysql.connector.connect(host=mysql_creds.host,
                                         database=mysql_creds.database,
                                         user=mysql_creds.user,
                                         password=mysql_creds.password)
        
        mySql_insert_query = create_division_insert_statement(division)

        cursor = connection.cursor()
        cursor.execute(mySql_insert_query)
        connection.commit()
        print("{}, inserted successfully into division table".format(division['name']))
        cursor.close()

    except mysql.connector.Error as error:
        print("Failed to insert record into Laptop table {}".format(error))

    finally:
        if connection.is_connected():
            connection.close()
#             print("MySQL connection is closed")



Atlantic, inserted successfully into division table
Central, inserted successfully into division table
Metropolitan, inserted successfully into division table
Pacific, inserted successfully into division table


## First I'm going to get information related to the teams

In [11]:
team_info =  requests.get(url='https://statsapi.web.nhl.com/api/v1/teams')
team_info = team_info.json()

### First, build the team table

In [12]:
try:
    connection = mysql.connector.connect(host=mysql_creds.host,
                                         database=mysql_creds.database,
                                         user=mysql_creds.user,
                                         password=mysql_creds.password)
    
    
    mySql_Create_Table_Query = """CREATE TABLE team(Id int NOT NULL,\
    fullName varchar(50) NOT NULL,\
    abbr varchar(5) NOT NULL,\
    teamName varchar(25) NOT NULL,\
    locationName varchar(25) NOT NULL,\
    division int NOT NULL,\
    conference int NOT NULL,\
    PRIMARY KEY (Id))"""
    
    
    cursor = connection.cursor()
    result = cursor.execute(mySql_Create_Table_Query)
    print("Team Table created successfully ")

except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

Team Table created successfully 
MySQL connection is closed


In [13]:
def create_team_insert_statement(team_stats):
    
    team_id = team_stats['id']
    team_fullname = team_stats['name']
    team_abbr = team_stats['abbreviation']
    team_teamName = team_stats['teamName']
    team_locationName = team_stats['locationName']
    team_division = team_stats['division']['id']
    team_conference = team_stats['conference']['id']
    
    insert_statement = """INSERT INTO team(Id, fullName, abbr, teamName, locationName, division, conference)\
    VALUES ({}, '{}', '{}', '{}', '{}', {}, {})"""\
    .format(team_id, team_fullname, team_abbr, team_teamName, team_locationName, team_division, team_conference)
    
    return insert_statement
    
for team in team_info['teams']:
        
    try:
        connection = mysql.connector.connect(host=mysql_creds.host,
                                         database=mysql_creds.database,
                                         user=mysql_creds.user,
                                         password=mysql_creds.password)

        mySql_insert_query = create_team_insert_statement(team)

        cursor = connection.cursor()
        cursor.execute(mySql_insert_query)
        connection.commit()
        print("{}, inserted successfully into team table".format(team['name']))
        cursor.close()

    except mysql.connector.Error as error:
        print("Failed to insert record into Laptop table {}".format(error))

    finally:
        if connection.is_connected():
            connection.close()
#             print("MySQL connection is closed")



New Jersey Devils, inserted successfully into team table
New York Islanders, inserted successfully into team table
New York Rangers, inserted successfully into team table
Philadelphia Flyers, inserted successfully into team table
Pittsburgh Penguins, inserted successfully into team table
Boston Bruins, inserted successfully into team table
Buffalo Sabres, inserted successfully into team table
Montréal Canadiens, inserted successfully into team table
Ottawa Senators, inserted successfully into team table
Toronto Maple Leafs, inserted successfully into team table
Carolina Hurricanes, inserted successfully into team table
Florida Panthers, inserted successfully into team table
Tampa Bay Lightning, inserted successfully into team table
Washington Capitals, inserted successfully into team table
Chicago Blackhawks, inserted successfully into team table
Detroit Red Wings, inserted successfully into team table
Nashville Predators, inserted successfully into team table
St. Louis Blues, inserted

### Game data

In [14]:
# # Loop over the counter and format the API call
# for i in range(1,6):
#     r = requests.get(url='http://statsapi.web.nhl.com/api/v1/game/'
#         + year + season_type +str(i).zfill(4)+'/feed/live')
#     data = r.json()
#     game_data.append(data)

In [15]:
# game_data[0]